<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Building-Agentic-AI-Systems/blob/main/Chapter_06.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chapter 6 – Exploring the Coordinator, Worker, and Delegator Approach
---

Install dependencies

In [ ]:
!pip install crewai langchain-openai

In [ ]:
import getpass
import os

api_key = getpass.getpass(prompt="Enter OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = api_key

### Role-based agents

Role-based agents within the CWD (Coordinator, Worker, and Delegator) model for a travel planner.

# CrewAI implementation
---

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown, HTML

llm = ChatOpenAI(model="gpt-4o")

<div class="alert alert-block alert-info"> 
<b>NOTE:</b> While we will use gpt-4o throughout this notebook, you can also use different LLMs for each of the agents. This is usually a recommended approach. For example for less complex tasks such as crafting a plan to book a travel itinerary, one could use a smaller model such as gpt-4o-mini, and for more complex tasks such as comparing travel options and reasoning a larger model is appropriate.
</div>


## Create the tools

In [ ]:
@tool("Search for available flights between cities")
def search_flights(origin: str, destination: str, date: str) -> dict:
    """
    Search for available flights between cities.
    
    Args:
        origin: Departure city
        destination: Arrival city
    
    Returns:
        Dictionary containing flight options and prices
    """
    # Emulate JSON data from an API
    return {
        "flights": [
            {"airline": "Air France", "price": 850, "departure": "New York (JFK)", "arrival": "Paris (CDG)", "duration": "7h 30m", "departure_time": "10:30 AM", "arrival_time": "11:00 PM"},
            {"airline": "Delta Airlines", "price": 780, "departure": "New York (JFK)", "arrival": "Paris (CDG)", "duration": "7h 45m", "departure_time": "5:30 PM", "arrival_time": "6:15 AM"},
            {"airline": "United Airlines", "price": 920, "departure": "New York (EWR)", "arrival": "Paris (CDG)", "duration": "7h 55m", "departure_time": "8:45 PM", "arrival_time": "9:40 AM"}
        ]}             

@tool("Find available hotels in a location") 
def find_hotels(location: str, check_in: str, check_out: str) -> dict:
    """
    Search for available hotels in a location.
    
    Args:
        location: City name
        check_in: Check-in date (YYYY-MM-DD)
        check_out: Check-out date (YYYY-MM-DD)
    
    Returns:
        Dictionary containing hotel options and prices
    """
    # Emulate JSON data from an API
    return {
        "hotels": [
            {"name": "Paris Marriott Champs Elysees", "price": 450, "check_in_date": check_in, "check_out_date": check_out, "rating": 4.5, "location": "Central Paris", "amenities": ["Spa", "Restaurant", "Room Service"]},
            {"name": "Citadines Saint-Germain-des-Prés", "price": 320, "check_in_date": check_in, "check_out_date": check_out, "rating": 4.2, "location": "Saint-Germain", "amenities": ["Kitchenette", "Laundry", "Wifi"]},
            {"name": "Ibis Paris Eiffel Tower", "price": 380, "check_in_date": check_in, "check_out_date": check_out, "rating": 4.0, "location": "Near Eiffel Tower", "amenities": ["Restaurant", "Bar", "Wifi"]}
        ]}

@tool("Find available activities in a location")
def find_activities(location: str, date: str, preferences: str) -> dict:
    """
    Find available activities in a location.
    
    Args:
        location: City name
        date: Activity date (YYYY-MM-DD)
        preferences: Activity preferences/requirements
        
    Returns:
        Dictionary containing activity options
    """
    # Implement actual activity search logic here
    return {
        "activities": [
            {"name": "Eiffel Tower Skip-the-Line", "description": "Priority access to the Eiffel Tower with guided tour of 1st and 2nd floors", "price": 65, "duration": "2 hours", "start_time": "10:00 AM", "meeting_point": "Eiffel Tower South Entrance"},
            {"name": "Louvre Museum Guided Tour", "description": "Expert-guided tour of the Louvre's masterpieces including Mona Lisa", "price": 85, "duration": "3 hours", "start_time": "2:00 PM", "meeting_point": "Louvre Pyramid"},
            {"name": "Seine River Dinner Cruise", "description": "Evening cruise along the Seine with 3-course French dinner and wine", "price": 120, "duration": "2.5 hours", "start_time": "7:30 PM", "meeting_point": "Port de la Bourdonnais"}
        ]}

@tool("Find local transportation options")
def find_transportation(location: str, origin: str, destination: str) -> dict:
    """
    Find local transportation options between locations.
    
    Args:
        location: City name
        origin: Starting point (e.g., "Airport", "Hotel", "Eiffel Tower")
        destination: End point (e.g., "City Center", "Museum", "Restaurant")
    
    Returns:
        Dictionary containing transportation options
    """
    # Return a simple JSON with transportation options
    return {
        "options": [
            { "type": "Metro", "cost": 1.90, "duration": "25 minutes", "frequency": "Every 5 minutes", "route": "Line 1 to Châtelet, then Line 4 to destination", "pros": "Fast, avoids traffic", "cons": "Can be crowded during peak hours"},
            { "type": "Taxi", "cost": 22.50, "duration": "20 minutes", "frequency": "On demand", "route": "Direct", "pros": "Door-to-door service, comfortable", "cons": "More expensive, subject to traffic"},
            { "type": "Bus", "cost": 1.90, "duration": "35 minutes", "frequency": "Every 10 minutes", "route": "Route 42 direct to destination", "pros": "Scenic route, above ground", "cons": "Slower than metro, subject to traffic"},
            { "type": "Walking", "cost": 0, "duration": "45 minutes", "frequency": "Anytime", "route": "Through city center", "pros": "Free, healthy, scenic", "cons": "Takes longer, weather dependent"}
        ],
        "passes": [
            { "name": "Day Pass", "cost": 7.50, "valid_for": "Unlimited travel for 24 hours", "recommended_if": "Making more than 4 trips in a day" },
            { "name": "Paris Visite",  "cost": 12.00, "valid_for": "Unlimited travel for 1 day, includes discounts to attractions", "recommended_if": "Planning to visit multiple tourist sites" }
        ]
    }

## Create the Agents

### Core Travel Workers

In [ ]:
flight_booking_worker = Agent(
    role="Flight Booking Specialist",
    goal="Find and book the optimal flights for the traveler",
    backstory="""You are an experienced flight booking specialist with extensive knowledge of airlines, 
    routes, and pricing strategies. You excel at finding the best flight options balancing cost, 
    convenience, and comfort according to the traveler's preferences.""",
    verbose=True,
    allow_delegation=False,
    tools=[search_flights],
    llm=llm,
    max_iter=1,
    max_retry_limit=3
)

hotel_booking_worker = Agent(
    role="Hotel Accommodation Expert",
    goal="Secure the ideal hotel accommodations for the traveler",
    backstory="""You have worked in the hospitality industry for over a decade and have deep knowledge 
    of hotel chains, boutique accommodations, and local lodging options worldwide. You're skilled at 
    matching travelers with accommodations that meet their budget, location preferences, and amenity requirements.""",
    verbose=True,
    allow_delegation=False,
    tools=[find_hotels],
    llm=llm,
    max_iter=1,
    max_retry_limit=3
)

activity_planning_worker = Agent(
    role="Activities and Excursions Planner",
    goal="Curate personalized activities and experiences for the traveler",
    backstory="""You're a well-traveled activities coordinator with insider knowledge of attractions, 
    tours, and unique experiences across numerous destinations. You're passionate about creating 
    memorable itineraries that align with travelers' interests, whether they seek adventure, culture, 
    relaxation, or culinary experiences.""",
    verbose=True,
    allow_delegation=False,
    tools=[find_activities],
    llm=llm,
    max_iter=1,
    max_retry_limit=3
)

transportation_worker = Agent(
    role="Local Transportation Coordinator",
    goal="Arrange efficient and convenient local transportation",
    backstory="""You specialize in local transportation logistics across global destinations. Your expertise 
    covers public transit systems, private transfers, rental services, and navigation, ensuring travelers 
    can move smoothly between destinations and activities.""",
    verbose=True,
    allow_delegation=False,
    tools=[find_transportation],
    llm=llm,
    max_iter=1,
    max_retry_limit=3
)

## Define tasks for all the CWD agents

### Tasks for the workers

In [ ]:
flight_search_task = Task(
    description="""
    Use the search_flights tool to find flight options from origin to destination.
    Review the returned JSON data and recommend the best option based on the traveler's priorities, if any.
    
    Compare the available options and recommended choice best meets their needs.
    """,
    agent=flight_booking_worker,
    expected_output="A flight itinerary for booking based on the traveler's preferences."
)

hotel_search_task = Task(
    description="""
    Use the find_hotels tool to search for accommodations in the destination.
    Review the returned JSON data and recommend the best option considering budget.
    
    Explain why your recommended choice is the best match for this traveler.
    """,
    agent=hotel_booking_worker,
    expected_output="A hotel recommendation based on the traveler's preferences and budget."
)

activity_planning_task = Task(
    description="""
    Use the find_activities tool to identify options in the destination for each day of the of the entire trip duration.
    The traveler's interests are: {activity_interests} with a {activity_pace} pace preference.
    
    Create a day-by-day plan using the returned JSON data, ensuring activities flow logically and match the traveler's interests.
    """,
    agent=activity_planning_worker,
    expected_output="A day-by-day activity plan that matches the traveler's interests and pace preferences."
)

transportation_planning_task = Task(
    description="""
    Use the find_transportation tool to identify options at the destination for:
    1. Airport to hotel transfer
    2. Transportation between daily activities
    3. Hotel to airport transfer
    
    Consider the traveler's preferences where possible.
    
    Based on the returned JSON data, recommend the best transportation options for each segment of their trip.
    """,
    agent=transportation_worker,
    expected_output="A transportation plan covering all necessary transfers during the trip."
)

### Defining the Coordinator Agent & Task

The `coordinate_request` function will use our Coordinator agent with a task to consume customer requests and craft a "plan" for the delegator agent later.

In [ ]:

coordinator_agent = Agent(
    role="Coordinator Agent",
    goal="Ensure cohesive travel plans and maintain high customer satisfaction",
    backstory="""A seasoned travel industry veteran with 15 years of experience in luxury travel planning 
    and project management. Known for orchestrating seamless multi-destination trips for high-profile clients 
    and managing complex itineraries across different time zones and cultures. 
    """,
    verbose=False,
    llm=llm,
    max_iter=1,
    max_retry_limit=3
)

In [ ]:
from textwrap import dedent

def coordinate_request(traveler_request):

    coordinator_to_delegator_task = Task(
        description=dedent(f"""\
        As the Coordinator Agent, you've received a travel planning request.
        
        Traveler request:
        {traveler_request}
        
        Create a clear, concise travel planning steps for this trip. Only plan
        for the things requested by the traveler, DO NOT assume or add things not requested. Provide a 
        short overview, followed by the steps required for flight booking, hotel booking, activities,
        and local transportation.
        
        Your output should be a step-by-step plan along with preference details that the Delegator Agent 
        can use to effectively assign tasks to the specialist workers. Do not provide any summary or mention 
        "Delegator" or "coordinator".
        """),
        expected_output="A detailed step-by-step travel plan for the delegator agent",
        agent=coordinator_agent
    )

    # Execute the coordinator's initial planning task
    coordinator_crew = Crew(
        agents=[coordinator_agent],
        tasks=[coordinator_to_delegator_task],
        verbose=False, # True if you want to see detailed execution
        process=Process.sequential
    )
    coordinator_plan = coordinator_crew.kickoff(inputs={'traveler_request': traveler_request})
    print("\n=== Coordinator Planning Complete ===\n")    
    return coordinator_plan


Test to see if our coordinator agent is creating a detailed plan for the delegator agent.

In [ ]:
request="""Traveler Alex Johnson is planning to travel to Paris from New York for his anniversary for 7 days and 2 people. 
- His total budget is about $8000, with hotel budget being $300.
- Direct flights preferred, morning departure if possible.
- Hotel in Paris under $400 with wifi preferred. Check in at 5/7/2025 and checkout at 5/14/2025
- Activities in paris should be moderate pace with some relaxation time built in
- Mix of walking and public transit, with occasional taxis for evening outings
"""
plan_for_delegator = coordinate_request(request)

View the plan crafted by the coordinator agent.

In [ ]:
display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Full step-by-step trip plan</h2></hr></div>'))
display(Markdown(plan_for_delegator.raw))

### Defining the Delegator Agent & Task

The `delegate_plan` function will use the travel plan crafted by the coordinator agent and subsequently delegate tasks to worker agents for each task (such book flight, book hotel etc.). It will also subsequently process the outputs of each worker agent and then craft a full itinerary for the traveler. Here, we use the `plan` generated by the `coordinator_agent` to craft a `goal` for the `delegator_agent`. 

We will use CrewAI's `manager_agent` feature to implement Delegator, which will manage the worker agents to search flights, search hotels, plan activities and look for local transportation using the respective worker agent.

In [ ]:
def delegate_plan(plan):
    delegator_goal=f"""
        Effectively distribute travel planning tasks to specialized workers to create a detailed booking itinerary
        for the plan below:
        
        {plan}
        
        Based on this plan, your goal is to create a detailed booking itinerary and trip plan for the user that includes
        flight booking & cost recommendation, hotels and hotel cost, activities and local transportation options
        and recommendations.
        """

    delegator_agent = Agent(
        role="Travel Planning Delegator",
        goal=delegator_goal,
        backstory="""You are an expert project manager with a talent for breaking down travel planning into 
        component tasks and assigning them to the right specialists. You understand each worker's strengths 
        and ensure they have the information needed to excel. You track progress, resolve bottlenecks, and 
        ensure all elements of the trip are properly addressed.""",    
        verbose=True,
        allow_delegation=True,
        llm=llm
    )
    # Execute the delegator's task assignment
    delegator_crew = Crew(
        agents=[flight_booking_worker, hotel_booking_worker, transportation_worker, activity_planning_worker],
        tasks=[flight_search_task, hotel_search_task, transportation_planning_task, activity_planning_task ],
        verbose=False,
        manager_agent=delegator_agent,
        process=Process.hierarchical,
        planning=True,        
        full_output=True
    )
    full_itinerary = delegator_crew.kickoff()
    print("\n=== Delegator Task Complete ===\n")
    return full_itinerary

<div class="alert alert-block alert-info"> 
<b>NOTE:</b> When you execute the following code cell you will see the full verbose execution of the Multi-agent delegator agent. You may also notice that at certain points the delegator failed to invoke the tool. This happens in case the LLM was unable to capture the required variables for the tool, at which point the CrewAI framework will retry the call by re-crafting it's inputs until it gets a proper tool call (often with smaller or cheaper LLMs). This is unfortunately one of the drawbacks of generic implementations, however with more custom implementations with CrewAI, you can steer the model to generate appropriate tool calls everytime given all the information is present.<br/>

Also note that the max_iter and max_retries_limit is set to 1 and 3 which means the agent will only be invoked once and will retry 3 times if there are errors. This means that the Agent may not come to a perfect answer with just 1 try, you may try to increast max_iter on the agents to experiment with the type of answers it produces.
</div>


In [ ]:
itinerary = delegate_plan(plan_for_delegator.raw)

In [ ]:
display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; Full travel plan – Report</h2></hr></div>'))

for task in itinerary.tasks_output:      
    display(Markdown(task.raw))